In [2]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [3]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/200images.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [11]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/dataset.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [4]:
Xtrain = []

In [12]:
# Get images
Xtrain = []
for filename in os.listdir('/content/dataset/Train/'):
  i=i+1
  if(i<200):
    Xtrain.append(img_to_array(load_img('/content/dataset/Train/'+filename)))
  else:
    break


for filename in os.listdir('/content/200images/'):
  Xtrain.append(img_to_array(load_img('/content/200images/'+filename)))

    
Xtrain = np.array(Xtrain, dtype=float)
Xtrain = 1.0/255*Xtrain

In [ ]:
inception = tf.keras.applications.InceptionResNetV2(
                include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
                pooling=None, classes=1000, classifier_activation='softmax'
                )



In [7]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [14]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 32

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

In [ ]:
#model.load_weights(0x7fc9a6226390)

In [ ]:
#Train model      
model.compile(optimizer='rmsprop', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), epochs=1000, steps_per_epoch=13)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/1000
13/13 [==============================] - 31s 2s/step - loss: 0.2910
Epoch 2/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0098
Epoch 3/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0095
Epoch 4/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0096
Epoch 5/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0096
Epoch 6/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0096
Epoch 7/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0096
Epoch 8/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0097
Epoch 9/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0097
Epoch 10/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0097
Epoch 11/1000
13/13 [==============================] - 27s 2s/step - loss: 0.0109
Epoch 12/1000
13/13 [==========

In [10]:
os.mkdir('result')

In [1]:
color_me = []
i=0
for filename in os.listdir('/content/dataset/Test/'):
  i=i+1
  if(i>20 and i<100):
    color_me.append(img_to_array(load_img('/content/dataset/Test/'+filename)))
  elif(i>100):
    break
  

  
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("/content/result/"+str(i)+".png", lab2rgb(cur))

NameError: ignored

In [3]:
from google.colab import files
for filename in os.listdir('/content/result/'):
  files.download('/content/result/'+filename) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
tf.keras.callbacks.ModelCheckpoint(filepath='/content',
                                                 save_weights_only=True,
                                                 verbose=1)